In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}",)

prompt = template.format(country_a = "Mexico", country_b="Thailand")

chat.predict(prompt)

In [ ]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Thailand",
)

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()

p.parse("Hello, how, are, you")



In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}.Do NOT reply with anything else.",
        ),
        ("human","{question}"),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

In [ ]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"
})

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human","I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat


In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a vegetarian chef... international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human","{recip}"),
])
# 3.4 영어 입력하기


veg_chain = veg_chef_prompt | chat

final_chain = chef_chain | veg_chain

final_chain.invoke({
    "cusine":"indian"
})

In [ ]:
from langchain.prompts.few_shot import PromptTemplate

t = PromptTemplate.from_template("What is the capital of {country}")

t.format(country="France")

In [ ]:
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)


t.format(country="France")

In [ ]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

    

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

example_template = """
    Human: {question}
    AI : {answer}
"""

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
    

example_prompt = PromptTemplate.from_template(example_template)
#example_prompt = PromptTemplate.from_template("Human: {question}\nAI : {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human : What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

# chain = prompt | chat

# chain.invoke({
#     "country":"Germany"
# })


In [ ]:
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

prompt.format(country="Germany")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {country}?"),
    ("ai","{answer}")
])



example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system","You are a geography expert"),
    example_prompt,
    ("human","What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country":"Germany"
})


In [ ]:
from langchain.prompts import load_prompt

prompta = load_prompt("./prompt.yaml")

prompta.format(country="Korea")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True,
)

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Express the movie title with 3 emoticons"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "What is three emojis of {movies}"),
    ]
)

examples = [
    {
        "movie":"what is three emojis of Top Gun",
        "answer":"""🛩️👨‍✈️🔥"""
    },
    {
        "movie":"what is three emojis of Top Godfather",
        "answer":"""👨‍👨‍👦🔫🍝"""
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human","what is three emojis of {movie}?"),
    ("ai","{answer}"),
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Express the movie title with 3 emoticons"),
        example_prompt,
        ("human", "what is three emojis of {movie}?"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie":movie})
    memory.save_context(
        {"input":movie},
        {"output":result.content},
    )
    print(result.content)

invoke_chain("Harry Potter And The Chamber Of Secrets")



In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredAPIFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import FAISS

cache_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(
    temperature=0.1,
)

splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredAPIFileLoader("./files/document.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


ValueError: Receive unexpected status code 401 from the API.

In [ ]:


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
    length_function=len,
)

loader = UnstructuredAPIFileLoader("../files/chapter_one.txt")


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

vector = embedder.embed_documents([
    "Hi",
    "how",
    "are",
    "you longer sentences because"
])